In [1]:
import numpy as np
from dependence.iterative_vines import get_pairs_by_levels, get_possible_structures
%load_ext autoreload
%autoreload 2

In [7]:
dim = 5
n_forced_pairs = 7
n_pairs = dim*(dim-1)/2
forced_pairs_ids = np.random.choice(range(n_pairs), n_forced_pairs, replace=False)
forced_pairs_ids = [4, 5, 2, 3]
pairs_by_levels = get_pairs_by_levels(dim, forced_pairs_ids)

Vine dimension: 5
Conditioning information:
	4 pairs with 0 conditionned variables
Pairs: [[4, 2], [4, 3], [3, 2], [4, 1]]
	3 pairs with 1 conditionned variables
Pairs: []
	2 pairs with 2 conditionned variables
Pairs: []
	1 pairs with 3 conditionned variables
Pairs: []
Concerned pairs: [[4, 2], [4, 3], [3, 2], [4, 1]]
Remaining pairs: [[2, 1], [3, 1], [5, 1], [5, 2], [5, 3], [5, 4]]


In [8]:
get_possible_structures(dim, pairs_by_levels)

[]